<a href="https://colab.research.google.com/github/mmender2/Aunalytics/blob/main/House.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow-recommenders


In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade category_encoders

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import os
import random
import os, sys, time, copy
from pathlib import Path
from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn import manifold
from sklearn import decomposition
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']



In [ ]:
# Mount Google Drive

train = pd.read_csv('/content/drive/MyDrive/House/train.csv')
test = pd.read_csv('/content/drive/MyDrive/House/test.csv')

X = train.drop(['Id', 'SalePrice'], axis=1)
y = train['SalePrice']

In [ ]:
# select numerical and categorical features
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Id' in num_cols:
    num_cols.remove('Id')
print(f"Total numerical features: {len(num_cols)}")
print(f"Total categorical features: {len(cat_cols)}")
print(num_cols)
print(cat_cols)

Total numerical features: 36
Total categorical features: 43
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fireplac

In [ ]:
std_scaler = preprocessing.StandardScaler()
mm_scaler = preprocessing.MinMaxScaler()
robust_scaler = preprocessing.RobustScaler()
quantile_scaler = preprocessing.QuantileTransformer(n_quantiles=60, output_distribution='normal')

In [ ]:
scaled_train_df = X.copy(deep=True)
scaled_test_df = test.copy(deep=True)
scaled_train_df[num_cols] = quantile_scaler.fit_transform(scaled_train_df[num_cols])
scaled_test_df[num_cols] = quantile_scaler.fit_transform(scaled_test_df[num_cols])

In [ ]:
trainDF = pd.concat([scaled_train_df, y], axis=1)

threshold = 0.2  # Set the threshold for the percentage of NaN values
column_nan_percentages = trainDF.isna().mean()  # Calculate the NaN percentages for each column

# Get the column names that have NaN percentages greater than the threshold
columns_to_drop = column_nan_percentages[column_nan_percentages > threshold].index

# Drop the columns from the DataFrame
df_dropped = trainDF.drop(columns=columns_to_drop)

# Print the DataFrame after dropping the columns
f = df_dropped.dropna()
weird_features = ['capital-gain', 'capital-loss']
categorical_cols = f.select_dtypes(include=['object', 'category']).columns.tolist()

# Encode categorical columns in X_train and X_test
train_encoded = pd.get_dummies(f.drop(['SalePrice'], axis=1), columns=categorical_cols)
#test_encoded = pd.get_dummies(XTest, columns=categorical_cols, prefix_sep='', prefix=None, drop_first=False)

# Ensure that both DataFrames have the same columns by aligning them
#train_encoded, test_encoded = train_encoded.align(test_encoded, axis=1, fill_value=0)
#imputer = SimpleImputer(strategy='most_frequent')  # You can choose different imputation strategies like 'mean', 'median', 'most_frequent', etc.

#X_train_encoded = imputer.fit_transform(train_encoded)
#X_test_encoded = imputer.transform(test_encoded)

In [ ]:
train_encoded.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.301922,-0.170767,-0.386465,0.609229,-0.415240,0.829574,0.583849,0.808776,0.664655,-5.199338,...,0,0,0,1,0,0,0,0,1,0
1,-5.199338,0.635007,0.063771,-0.021244,1.826356,0.063771,-0.369353,-5.199338,1.108151,-5.199338,...,0,0,0,1,0,0,0,0,1,0
2,0.301922,-0.040529,0.575610,0.609229,-0.415240,0.715053,0.485786,0.661215,0.179793,-5.199338,...,0,0,0,1,0,0,0,0,1,0
3,0.687885,-0.509832,0.023611,0.609229,-0.415240,-1.678650,-0.558840,-5.199338,-0.254601,-5.199338,...,0,0,0,1,1,0,0,0,0,0
4,0.301922,0.799969,1.259287,1.271953,-0.415240,0.661215,0.346704,1.367282,0.534175,-5.199338,...,0,0,0,1,0,0,0,0,1,0


In [ ]:


# Define the number of splits and create the TimeSeriesSplit object
n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

# Cross validation
v = 0
for trainIndex, testIndex in tscv.split(train_encoded):
    # Split data based on indices
    XTrain, XTest = train_encoded.iloc[trainIndex], train_encoded.iloc[testIndex]
    yTrain, yTest = y.iloc[trainIndex], y.iloc[testIndex]

    # Reshape the data for LSTM
    timeSteps = 1
    features = train_encoded.shape[1]
    reshapeTrain = np.array(XTrain)
    reshapeTest = np.array(XTest)
    reshapeTrainY = np.array(yTrain)
    reshapeTestY = np.array(yTest)

    # Reshape the arrays for LSTM input
    XTrainReshaped = reshapeTrain.astype('float32').reshape(len(reshapeTrain), timeSteps, features)
    XTestReshaped = reshapeTest.astype('float32').reshape(len(reshapeTest), timeSteps, features)
    yTrainReshaped = reshapeTrainY.astype('float32').reshape(len(reshapeTrainY), timeSteps, 1)
    yTestReshaped = reshapeTestY.astype('float32').reshape(len(reshapeTestY), timeSteps, 1)

    # Define the LSTM model
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(timeSteps, features)),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse')



In [ ]:

rows_with_nans = scaled_train_df.isna().sum(axis=1).sum()
print("Total rows with NaN values:", rows_with_nans)

Total rows with NaN values: 6965


In [ ]:
num_rows = len(scaled_train_df)
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 1460


In [ ]:
# Assuming your DataFrame is named df
threshold = 0.2  # Set the threshold for the percentage of NaN values
column_nan_percentages = scaled_train_df.isna().mean()  # Calculate the NaN percentages for each column

# Get the column names that have NaN percentages greater than the threshold
columns_to_drop = column_nan_percentages[column_nan_percentages > threshold].index

# Drop the columns from the DataFrame
df_dropped = scaled_train_df.drop(columns=columns_to_drop)

# Print the DataFrame after dropping the columns
f = df_dropped.dropna()

In [ ]:
f = df_dropped.dropna()
print(len(f))

1094


# Two Tower Model

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Assuming you have the following columns in X_train:
# numerical_features: numpy array of shape (num_samples, num_numerical_features)
# categorical_features: numpy array of shape (num_samples, num_categorical_features)

# Define the number of numerical and categorical features
num_numerical_features = X[num_cols].shape[1]
num_categorical_features = X[cat_cols].shape[1]

# Define the input layers for numerical and categorical features
input_numerical = Input(shape=(num_numerical_features,), name='numerical_input')
input_categorical = Input(shape=(num_categorical_features,), name='categorical_input')

# Add embedding layer for categorical features if necessary
# For example, if a categorical feature has many unique categories, you can use embedding to represent them
# embedding_layer = Embedding(input_dim=num_categories, output_dim=embedding_dim)(input_categorical)
# flattened_categorical = Flatten()(embedding_layer)

# Alternatively, you can use one-hot encoding for categorical features
encoder = OneHotEncoder()
encoded_categorical_features = encoder.fit_transform(test[cat_cols]).toarray()

# Concatenate numerical and categorical features
concatenated = Concatenate()([input_numerical, encoded_categorical_features])



In [ ]:
class Playlist_Tower(tf.keras.Model):
  def __init__(self, layer_sizes):
    super().__init__()

    self.lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(layer_sizes[0], input_shape=layer_sizes[1:]),
            tf.keras.layers.Dense(units=32, activation='relu'),
        ])
  def call(self, data):
        # Call the LSTM model with the input data
    return self.lstm_model(data)



In [ ]:
class Candidate_Track_Tower(tf.keras.Model):
    def __init__(self, layer_sizes):
        super().__init__()

        # Define the LSTM model and assign it to a class attribute
        self.lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(layer_sizes[0], input_shape=layer_sizes[1:]),  # Use your desired number of units for LSTM
            tf.keras.layers.Dense(units=32, activation='relu'),
        ])

    def call(self, data):
        # Call the LSTM model with the input data
        return self.lstm_model(data)


In [ ]:
class QueryCandidateDataset(tf.data.Dataset):
    def __init__(self, query_data, candidate_data):
        self.query_data = query_data
        self.candidate_data = candidate_data

    def _generator(self):
        for query, candidate in zip(self.query_data, self.candidate_data):
            yield query, candidate

    def __iter__(self):
        return self._generator()

    def __len__(self):
        return len(self.query_data)

    def element_spec(self):
        return (tf.TensorSpec(shape=self.query_data.shape[1:], dtype=self.query_data.dtype),
                tf.TensorSpec(shape=self.candidate_data.shape[1:], dtype=self.candidate_data.dtype))

    def batch(self, batch_size):
        return self

def generator_wrapper(query_data, candidate_data):
    dataset = QueryCandidateDataset(query_data, candidate_data)
    for data in dataset:
        yield data

In [ ]:
class TheTwoTowers(tfrs.models.Model):
    def __init__(self, layer_sizes, vocab_dict, parsed_candidate_dataset):
        super().__init__()

        self.query_tower = Playlist_Tower(layer_sizes)
        self.candidate_tower = Candidate_Track_Tower(layer_sizes)

        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=parsed_candidate_dataset.batch(128).map(
                    self.candidate_tower,
                    num_parallel_calls=tf.data.AUTOTUNE
                ).prefetch(tf.data.AUTOTUNE)
            )
        )

    def compute_loss(self, data, training=False):
        query_data, candidate_data, candidate_ids = data  # Unpack the custom dataset

        query_embeddings = self.query_tower(query_data)
        candidate_embeddings = self.candidate_tower(candidate_data)

        # Concatenate the embeddings from both towers
        all_embs = tf.concat([query_embeddings, candidate_embeddings], axis=1)

        return self.task(
            query_embeddings,
            candidate_embeddings,
            compute_metrics=not training,
            candidate_ids=candidate_ids,
            compute_batch_metrics=True
        )

    def call(self, data):
        query_data, candidate_data, candidate_ids = data
        query_embeddings = self.query_tower(query_data)
        candidate_embeddings = self.candidate_tower(candidate_data)

        # Concatenate the embeddings from both towers
        all_embs = tf.concat([query_embeddings, candidate_embeddings], axis=1)

        return all_embs, candidate_ids


In [ ]:
# Compile the model and train it using appropriate data
# Compile the model
# Convert NumPy arrays to TensorFlow Tensors
#two_towers_model.compile(optimizer='adam', loss='mse', weighted_metrics=['mae', 'mse'])
#y_test_reshaped = yTestReshaped.reshape(yTestReshaped.shape[0], 5)

# Assuming you have X_train and y_train ready
#history = two_towers_model.fit(custom_dataset, epochs=50, verbose=1, validation_data=custom_datasetTest)


In [ ]:
"""# Access loss values from history object
loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot training and validation loss
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
"""

"# Access loss values from history object\nloss = history.history['loss']\nval_loss = history.history['val_loss']\n\n# Plot training and validation loss\nepochs = range(1, len(loss) + 1)\nplt.plot(epochs, loss, 'b', label='Training Loss')\nplt.plot(epochs, val_loss, 'r', label='Validation Loss')\nplt.title('Training and Validation Loss')\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\nplt.legend()\nplt.show()\n"

# Attempt 2

In [ ]:
class HouseFeatureTower(tf.keras.layers.Layer):
    def __init__(self, layer_sizes):
        super().__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=64, input_shape=(layer_sizes[0], layer_sizes[1])),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu"),
        ])

    def call(self, inputs):
        return self.encoder(inputs)

In [ ]:
class SalePriceTower(tf.keras.layers.Layer):
    def __init__(self, layer_sizes):
        super().__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=64, input_shape=(layer_sizes[0], layer_sizes[1])),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu"),
        ])

    def call(self, inputs):
        return self.encoder(inputs)

In [ ]:
class HousePricePredictionModel(tfrs.models.Model):
    def __init__(self, layer_sizes):
        super().__init__()
        self.query_tower = HouseFeatureTower(layer_sizes)
        self.candidate_tower = SalePriceTower(layer_sizes)

    def compute_loss(self, data, training=False):
        query_embeddings = self.query_tower(data[0])
        candidate_embeddings = self.candidate_tower(data[1])
        # Normalize the embeddings (optional but recommended for cosine similarity)
        query_embeddings = tf.nn.l2_normalize(query_embeddings, axis=1)
        candidate_embeddings = tf.nn.l2_normalize(candidate_embeddings, axis=1)
        # Calculate cosine similarity between query and candidate embeddings
        cosine_similarities = tf.reduce_sum(query_embeddings * candidate_embeddings, axis=1)

        # Compute the pairwise hinge loss
        loss = tf.maximum(0.0, 1.0 - cosine_similarities)

        # Compute the mean loss over the batch
        #loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(candidate_embeddings, cosine_similarities, from_logits=True))
        # Compute the mean loss over the batch
        loss = tf.reduce_mean(loss)

        # Compute the indices of the top 5 most similar candidates for each query
        _, indices = tf.math.top_k(cosine_similarities, k=1, sorted=True)

        # Get the true labels for the top-k candidates for each query
        true_labels = tf.gather(data[1], indices)  # Shape: (batch_size, k)

        # Check if the true labels match the actual query labels
        query_labels = tf.expand_dims(data[1], axis=1)  # Shape: (batch_size, 1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(true_labels, query_labels), tf.float32))

        return loss, accuracy
    def train_step(self, data):
        # Unpack the data
        query_data, candidate_data = data

        with tf.GradientTape() as tape:
            # Compute predictions
            query_embeddings = self.query_tower(query_data, training=True)
            candidate_embeddings = self.candidate_tower(candidate_data, training=True)

            # Compute the loss value
            # (you need to define this according to your loss function)
            loss_value = self.compute_loss(query_embeddings, candidate_embeddings)

        # Compute gradients and apply updates
        gradients = tape.gradient(loss_value, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {"loss": loss_value}

In [ ]:
cat_cols = XTrain.select_dtypes(include=['object', 'category']).columns.tolist()
time_steps = 3
num_features = XTrain.shape[1]

# Reshape XTrain to create X_train_reshaped
XTrainReshaped = np.array([XTrain.iloc[i:i + time_steps].values for i in range(len(XTrain) - time_steps + 1)])
XTestReshaped = np.array([XTest.iloc[i:i + time_steps].values for i in range(len(XTest) - time_steps + 1)], dtype=object)
yTrainReshaped = np.array([yTrain.iloc[i:i + time_steps].values for i in range(len(yTrain) - time_steps + 1)])
yTestReshaped = np.array([yTest.iloc[i:i + time_steps].values for i in range(len(yTest) - time_steps + 1)])
# reshape the array
"""XTrainReshaped = np.array(XTrain.values.astype('float32').reshape(len(XTrain), time_steps, XTrain.shape[1]))
XTestReshaped = np.array(XTest.values.astype('float32').reshape(len(XTest), time_steps, XTest.shape[1]))
yTrainReshaped = np.array(yTrain.values.astype('float32').reshape(len(yTrain), time_steps, 1))
yTestReshaped = np.array(yTest.values.astype('float32').reshape(len(yTest), time_steps, 1))"""
yTrainReshaped_3d = yTrainReshaped.reshape(yTrainReshaped.shape[0], yTrainReshaped.shape[1], 1)
yTrainReshaped_3d = np.repeat(yTrainReshaped_3d, num_features, axis=2)
yTestReshaped_3d = yTestReshaped.reshape(yTestReshaped.shape[0], yTestReshaped.shape[1], 1)
yTestReshaped_3d = np.repeat(yTestReshaped_3d, num_features, axis=2)

In [ ]:
num_candidates = len(XTrainReshaped)
X_train_reshaped = XTrainReshaped.astype(np.int32)
yTrainReshaped_3d = yTrainReshaped_3d.astype(np.int32)
candidate_ids_list = list(range(1, num_candidates + 1))
candidate_ids_tensor = tf.constant(candidate_ids_list, dtype=tf.int32)

num_candidates = len(XTestReshaped)
X_test_reshaped = XTestReshaped.astype(np.float32)
yTestReshaped_3d = yTestReshaped_3d.astype(np.float32)
candidate_ids_list2 = list(range(1, num_candidates + 1))
candidate_ids_tensorT = tf.constant(candidate_ids_list2, dtype=tf.int32)

parsed_candidate_dataset = tf.data.Dataset.from_tensor_slices(yTrainReshaped_3d)
vocab_dict = tf.data.Dataset.from_tensor_slices(X_train_reshaped)

# Reshape X_train_reshaped to add a new dimension for time steps
#X_train_reshaped_3d = X_train_reshaped[:, tf.newaxis, :]

# Now the shape of X_train_reshaped_3d will be: [batch_size, time_steps=1, num_features]

# For yTrainReshaped_3d, if you want to keep it as a three-dimensional tensor
# with time_steps=1, you can add a new dimension as well
#yTrainReshaped_3d = yTrainReshaped_3d[:, tf.newaxis, :]

# Now the shape of yTrainReshaped_3d will be: [batch_size, time_steps=1, num_candidates]

# Similarly, do the same for X_test_reshaped and yTestReshaped_3d
#X_test_reshaped_3d = X_test_reshaped[:, tf.newaxis, :]
#yTestReshaped_3d = yTestReshaped_3d[:, tf.newaxis, :]
custom_dataset = tf.data.Dataset.from_tensor_slices(
    (
        X_train_reshaped,
        tf.cast(yTrainReshaped_3d, tf.int32)
    )
)

custom_datasetTest = tf.data.Dataset.from_tensor_slices(
    (
        X_test_reshaped,
        tf.cast(yTestReshaped_3d, tf.int32)
    )
)

#two_towers_model = TheTwoTowers(layer_sizes=[64, time_steps, num_features], vocab_dict=vocab_dict, parsed_candidate_dataset=parsed_candidate_dataset)


In [ ]:
# Instantiate the model
layer_sizes=[time_steps, num_features]
model = HousePricePredictionModel(layer_sizes)

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
# Train the model
model.fit(X_train_reshaped, yTrainReshaped_3d,batch_size=32, epochs=50, verbose=1, validation_data=(X_test_reshaped,yTestReshaped_3d))


Epoch 1/50


ValueError: ignored

In [ ]:
print(X_train_reshaped.shape)

In [ ]:
print(custom_dataset)


# LSTM

In [ ]:
import os, sys, time, copy
from pathlib import Path
from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn import manifold
from sklearn import decomposition
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
#from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce


mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def summary(df):
    print(f"Data shape: {df.shape}")
    print("\nSummary of columns missing values")
    print(df[df.isna().sum()[df.isna().sum() > 0].index].dtypes)
    print()
    summ = pd.DataFrame(df.dtypes, columns=['Data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values

    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first quartile'] = desc.loc[:, '25%'].values
    summ['second quartile'] = desc.loc[:, '50%'].values
    summ['third quartile'] = desc.loc[:, '75%'].values

    return summ

In [ ]:
summary(train)

Data shape: (1460, 81)

Summary of columns missing values
LotFrontage     float64
Alley            object
MasVnrType       object
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
FireplaceQu      object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
PoolQC           object
Fence            object
MiscFeature      object
dtype: object



,Data type,#missing,%missing,#unique,min,max,first quartile,second quartile,third quartile
Id,int64,0,0.000000,1460,1.0,1460.0,365.75,730.5,1095.25
MSSubClass,int64,0,0.000000,15,20.0,190.0,20.0,50.0,70.0
MSZoning,object,0,0.000000,5,NaN,NaN,NaN,NaN,NaN
LotFrontage,float64,259,17.739726,110,21.0,313.0,59.0,69.0,80.0
LotArea,int64,0,0.000000,1073,1300.0,215245.0,7553.5,9478.5,11601.5
...,...,...,...,...,...,...,...,...,...
MoSold,int64,0,0.000000,12,1.0,12.0,5.0,6.0,8.0
YrSold,int64,0,0.000000,5,2006.0,2010.0,2007.0,2008.0,2009.0
SaleType,object,0,0.000000,9,NaN,NaN,NaN,NaN,NaN
SaleCondition,object,0,0.000000,6,NaN,NaN,NaN,NaN,NaN


In [ ]:
# select numerical and categorical features
num_cols = train.select_dtypes(include=np.number).columns.tolist()
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Id' in cat_cols:
    cat_cols.remove('Id')
print(f"Total numerical features: {len(num_cols)}")
print(f"Total categorical features: {len(cat_cols)}")
print(num_cols)
print(cat_cols)

Total numerical features: 38
Total categorical features: 43
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Fun

In [ ]:
std_scaler = preprocessing.StandardScaler()
mm_scaler = preprocessing.MinMaxScaler()
robust_scaler = preprocessing.RobustScaler()
quantile_scaler = preprocessing.QuantileTransformer(n_quantiles=60, output_distribution='uniform')

In [ ]:
#num_cols.remove('Id')
scaled_train_df = train.copy(deep=True)
scaled_test_df = test.copy(deep=True)
scaled_train_df[num_cols] = quantile_scaler.fit_transform(scaled_train_df[num_cols])
#num_cols.remove('SalePrice')
scaled_test_df[num_cols] = quantile_scaler.fit_transform(scaled_test_df[num_cols])

KeyError: ignored

In [ ]:
print(num_cols)

In [ ]:
import tensorflow.keras.backend as K
# RMSE calculation
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
X = scaled_train_df.drop('SalePrice', axis=1)
y = scaled_train_df['SalePrice']
X = X.dropna(axis=1)
# select numerical and categorical features
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Define the number of splits and create the TimeSeriesSplit object
n_splits = 3
tscv = TimeSeriesSplit(n_splits=n_splits)
time_steps = 3
"""encoder = ce.OrdinalEncoder(handle_unknown='return_nan')
encoder.fit(scaled_train_df[cat_cols])
  # Encode categorical columns in X_train
  # Transform the train and test data
X[cat_cols] = encoder.transform(X[cat_cols])"""
label_encoder = LabelEncoder()
for col in cat_cols:
    X[col] = label_encoder.fit_transform(X[col])
num_features = X.shape[1]
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(time_steps, num_features), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1)
])
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[rmse])
# Cross validation
v = 4
for trainIndex, testIndex in tscv.split(X):
  random.seed(v)
  XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3)

  # Reshape the data for LSTM
  samplesTrain, _ = XTrain.shape
  samplesTest, _ = XTest.shape
  reshapeTrain = array(XTrain)
  reshapeTest = array(XTest)
  reshapeTrainY = array(yTrain)
  reshapeTestY = array(yTest)

  # reshape the array
  XTrainReshaped = np.array([XTrain.iloc[i:i + time_steps].values for i in range(len(XTrain) - time_steps + 1)])
  XTestReshaped = np.array([XTest.iloc[i:i + time_steps].values for i in range(len(XTest) - time_steps + 1)])
  yTrainReshaped = np.array([yTrain.iloc[i:i + time_steps].values for i in range(len(yTrain) - time_steps + 1)])
  yTestReshaped = np.array([yTest.iloc[i:i + time_steps].values for i in range(len(yTest) - time_steps + 1)])

  """XTrainReshaped = reshapeTrain.astype('float32').reshape(len(XTrainReshaped), time_steps, num_features)
  XTestReshaped = reshapeTest.astype('float32').reshape(len(XTestReshaped), time_steps, num_features)
  yTrainReshaped = reshapeTrainY.astype('float32').reshape(len(yTrainReshaped), num_features, 1)
  yTestReshaped = reshapeTestY.astype('float32').reshape(len(yTestReshaped), num_features, 1)
"""
  # Train the model
  history = model.fit(XTrainReshaped, yTrainReshaped, epochs=100, verbose=1, validation_data=(XTestReshaped, yTestReshaped))
  v += 1


Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 0.1196 - rmse: 0.3382

KeyboardInterrupt: ignored

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.title('RMSE Model')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
print(history.history.keys())